In [ ]:
import os, glob, json, csv, subprocess, sys, re
from git import *
from subprocess import Popen, PIPE
from os import path
from datetime import datetime
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import plotly.graph_objs as go
import plotly.plotly as ply
import plotly
from plotly.offline import plot, init_notebook_mode, iplot

%matplotlib inline

# Defining repository

In [ ]:
userhome = os.path.expanduser('~')
repository = userhome + r'/different-diff/dataset/hbase/'

In [ ]:
analyze_dir = userhome + r'/different-diff/analyze/analyze_hbase/'

# Opening the earliest affected version

In [ ]:
affected_version = pd.read_csv(analyze_dir + 'step_3/affected_version/earliest_version.csv')
cols = ['bug_id','earliest_affected_version','date_release']
aff = affected_version[cols]
aff

In [ ]:
#Combining data histogram with earliest versions
filedata_hist = pd.read_csv(analyze_dir + 'file-diff/number_of_files/histogram_total_bugline.csv')
filedata_hist = filedata_hist[['bug_id', 'filepath', 'number_of_buggyline']]

details_hist = filedata_hist.join(aff.set_index('bug_id')[['earliest_affected_version','date_release']], on='bug_id')
#details_hist    

In [ ]:
#Combining data minimal with earliest versions
filedata_min = pd.read_csv(analyze_dir + 'file-diff/number_of_files/minimal_total_bugline.csv')
filedata_min = filedata_min[['bug_id', 'filepath', 'number_of_buggyline']]

details_min = filedata_min.join(aff.set_index('bug_id')[['earliest_affected_version','date_release']], on='bug_id')
#details_min

In [ ]:
#Combining data myers with earliest versions
filedata_myers = pd.read_csv(analyze_dir + 'file-diff/number_of_files/myers_total_bugline.csv')
filedata_myers = filedata_myers[['bug_id', 'filepath', 'number_of_buggyline']]

details_myers = filedata_myers.join(aff.set_index('bug_id')[['earliest_affected_version','date_release']], on='bug_id')
#details_myers

In [ ]:
#Combining data patience with earliest versions
filedata_pat = pd.read_csv(analyze_dir + 'file-diff/number_of_files/patience_total_bugline.csv')
filedata_pat = filedata_pat[['bug_id', 'filepath', 'number_of_buggyline']]

details_pat = filedata_pat.join(aff.set_index('bug_id')[['earliest_affected_version','date_release']], on='bug_id')
details_pat

# Start comparing buglines with the blamed files

In [ ]:
columns = ['parent_id', 'filename', 'filepath']

# Histogram

In [ ]:
filelist_hist = glob.glob(analyze_dir + 'step_3/blame_files/blame_histogram/csv/*')
blamefiles_hist = ';'.join(filelist_hist).split(';')
blamefiles_hist.sort()
print ("Found " + str(len(blamefiles_hist)) + " files") 

In [ ]:
filebug_hist = glob.glob(analyze_dir + 'file-diff/buggylines/histogram/*')
diff_files_hist = ';'.join(filebug_hist).split(';')
diff_files_hist.sort()
print ("Found " + str(len(diff_files_hist)) + " files")

In [ ]:
h = 2312
print (blamefiles_hist[h])
print ("\n" + diff_files_hist[h])

In [ ]:
df_algo_files_hist = []
x = 0
while x < len(diff_files_hist):
    sys.stdout.write("\rExtracting file %i" %(x+1))
    sys.stdout.flush()
    df_algo_files_hist.append([diff_files_hist[x]])
    x += 1

df_algo_files_hist

In [ ]:
#Open csv file from result of git blame for <filename>.java
for num, blame in enumerate(blamefiles_hist):
    sys.stdout.write('\rComparing bugline_files and blame_files: %i' % (num+1) + ' out of %i' % (len(blamefiles_hist)))
    sys.stdout.flush()
    
    fields = ['line_number', 'code', 'date']
    blamefile = pd.read_csv(blame)
    blamefile = blamefile[fields]

    origin_code = []
    for n in range(0, len(blamefile)):
        temp = [blamefile.iloc[n][0], str(blamefile.iloc[n][1]).replace('nan',''), blamefile.iloc[n][2]]
        origin_code.append(temp)
    
    #Open diff_file <filename>.java-HBASE-xxxx_<diffalgorithm>buglines.diff vs blame_<filename>
    for m, df in enumerate(df_algo_files_hist[num]):
        dfname = re.search(("(?:\w+[-]\w+[-]\w+|\w+[-]\w+|\w+)[.]\w.*"), df)
        dfname = dfname.group(0)
        
        diff_file = open(analyze_dir + 'file-diff/buggylines/histogram/' + dfname,'r')
        bugline = diff_file.read().split('\n')

        if bugline[len(bugline)-1] == '':
            for i in range(0,len(bugline)-1):
                bugline[i] = bugline[i][1:]
            del bugline[len(bugline)-1]
        else:
            for i in range(0,len(bugline)):
                bugline[i] = bugline[i][1:]
        
        bug_id = re.compile('(HBASE-[0-9]*)')
        bug_id = bug_id.findall(df)
        
        for vers in range(0,len(aff)):
            if bug_id[0] == aff.iloc[vers][0]:
                version = aff.iloc[vers][1]
                daterelease = aff.iloc[vers][2]
                break;
            else:
                version = 'None'
                daterelease = 'None'
        
        num = 0
        blame_compare = []
        org_code = [el[1] for el in origin_code]
        for y, bugs in enumerate(bugline):
            if bugs in org_code:
                for x in range(num, len(origin_code)):                
                    codex = origin_code[x][1]
                    if bugs == codex:
                        if version != 'None':
                            if origin_code[x][2] <= daterelease:
                                flag = 'bug-introducing change'
                            else:
                                flag = 'incorrect'
                        else:
                            flag = 'unknown'
                        temp = [bug_id[0], origin_code[x][0], bugs, "True", origin_code[x][2], version, daterelease, flag]
                        blame_compare.append(temp)
                        num = origin_code[x][0]
                        break;
            else:
                temp = [bug_id[0], 'None', bugs, "False", 'None', version, daterelease, 'unknown']
                blame_compare.append(temp)
                #break;
        
        fnames = re.search(("(?:\w+[-]\w+[-]\w+|\w+[-]\w+|\w+)[.]\w+[_]|(?<=histogram\/)(\w+[_])(?<=[_])"), df)
        fnames = fnames.group(0)
        
        #algoname = re.search(("[_]\w+.diff"), df)
        #algoname = algoname.group(0)
        
        blamename = re.search(("(?:\w+[-]\w+[-]\w+|\w+[-]\w+|\w+)[.]\w.*"), blame)
        blamename = blamename.group(0)
        
        with open (analyze_dir + r'step_3/diff-file_blame-file_comparison/histogram_comparison/diff_' + fnames + "histogram-" + blamename, 'w') as csvfile:
            writers = csv.writer(csvfile, delimiter = ",")
            header = ['bug_id', 'line_number', 'buggy_code', 'does_bugline_from_originfile?', 'date_added_in_originfile', 'earliest_affected_version', 'version_release_date', 'bug_class']
            writers.writerow(header)
            for item in blame_compare:
                writers.writerow(item)
                
print ('\nComparing buglines are complete')

# Minimal

In [ ]:
filelist_min = glob.glob(analyze_dir + 'step_3/blame_files/blame_minimal/csv/*')
blamefiles_min = ';'.join(filelist_min).split(';')
blamefiles_min.sort()
print ("Found " + str(len(blamefiles_min)) + " files")

In [ ]:
filebug_min = glob.glob(analyze_dir + 'file-diff/buggylines/minimal/*')
diff_files_min = ';'.join(filebug_min).split(';')
diff_files_min.sort()
print ("Found " + str(len(diff_files_min)) + " files")

In [ ]:
h = 1763
print (blamefiles_min[h])
print ("\n" + diff_files_min[h])

In [ ]:
df_algo_files_min = []
x = 0
while x < len(diff_files_min):
    sys.stdout.write("\rExtracting file %i" %(x+1))
    sys.stdout.flush()
    df_algo_files_min.append([diff_files_min[x]])
    x += 1
    
df_algo_files_min

In [ ]:
#Open csv file from result of git blame for <filename>.java
for num, blame in enumerate(blamefiles_min):
    sys.stdout.write('\rComparing bugline_files and blame_files: %i' % (num+1) + ' out of %i' % (len(blamefiles_min)))
    sys.stdout.flush()
    
    fields = ['line_number', 'code', 'date']
    blamefile = pd.read_csv(blame)
    blamefile = blamefile[fields]

    origin_code = []
    for n in range(0, len(blamefile)):
        temp = [blamefile.iloc[n][0], str(blamefile.iloc[n][1]).replace('nan',''), blamefile.iloc[n][2]]
        origin_code.append(temp)
    
    #Open diff_file <filename>.java-HBASE-xxxx_<diffalgorithm>buglines.diff vs blame_<filename>
    for m, df in enumerate(df_algo_files_min[num]):
        dfname = re.search(("(?:\w+[-]\w+[-]\w+|\w+[-]\w+|\w+)[.]\w.*"), df)
        dfname = dfname.group(0)
        
        diff_file = open(analyze_dir + 'file-diff/buggylines/minimal/' + dfname,'r')
        bugline = diff_file.read().split('\n')

        if bugline[len(bugline)-1] == '':
            for i in range(0,len(bugline)-1):
                bugline[i] = bugline[i][1:]
            del bugline[len(bugline)-1]
        else:
            for i in range(0,len(bugline)):
                bugline[i] = bugline[i][1:]
        
        bug_id = re.compile('(HBASE-[0-9]*)')
        bug_id = bug_id.findall(df)
        
        for vers in range(0,len(aff)):
            if bug_id[0] == aff.iloc[vers][0]:
                version = aff.iloc[vers][1]
                daterelease = aff.iloc[vers][2]
                break;
            else:
                version = 'None'
                daterelease = 'None'
        
        num = 0
        blame_compare = []
        org_code = [el[1] for el in origin_code]
        for y, bugs in enumerate(bugline):
            if bugs in org_code:
                for x in range(num, len(origin_code)):                
                    codex = origin_code[x][1]
                    if bugs == codex:
                        if version != 'None':
                            if origin_code[x][2] <= daterelease:
                                flag = 'bug-introducing change'
                            else:
                                flag = 'incorrect'
                        else:
                            flag = 'unknown'
                        temp = [bug_id[0], origin_code[x][0], bugs, "True", origin_code[x][2], version, daterelease, flag]
                        blame_compare.append(temp)
                        num = origin_code[x][0]
                        break;
            else:
                temp = [bug_id[0], 'None', bugs, "False", 'None', version, daterelease, 'unknown']
                blame_compare.append(temp)
                #break;
        
        fnames = re.search(("(?:\w+[-]\w+[-]\w+|\w+[-]\w+|\w+)[.]\w+[_]|(?<=minimal\/)(\w+[_])(?<=[_])"), df)
        fnames = fnames.group(0)
        
        #algoname = re.search(("[_]\w+.diff"), df)
        #algoname = algoname.group(0)
        
        blamename = re.search(("(?:\w+[-]\w+[-]\w+|\w+[-]\w+|\w+)[.]\w.*"), blame)
        blamename = blamename.group(0)
        
        with open (analyze_dir + r'step_3/diff-file_blame-file_comparison/minimal_comparison/diff_' + fnames + "minimal-" + blamename, 'w') as csvfile:
            writers = csv.writer(csvfile, delimiter = ",")
            header = ['bug_id', 'line_number', 'buggy_code', 'does_bugline_from_originfile?', 'date_added_in_originfile', 'earliest_affected_version', 'version_release_date', 'bug_class']
            writers.writerow(header)
            for item in blame_compare:
                writers.writerow(item)
                
print ('\nComparing buglines are complete')

# Myers

In [ ]:
filelist_myers = glob.glob(analyze_dir + 'step_3/blame_files/blame_myers/csv/*')
blamefiles_myers = ';'.join(filelist_myers).split(';')
blamefiles_myers.sort()
print ("Found " + str(len(blamefiles_myers)) + " files") 

In [ ]:
filebug_myers = glob.glob(analyze_dir + 'file-diff/buggylines/myers/*')
diff_files_myers = ';'.join(filebug_myers).split(';')
diff_files_myers.sort()
print ("Found " + str(len(diff_files_myers)) + " files")

In [ ]:
h = 820
print (blamefiles_myers[h])
print ("\n" + diff_files_myers[h])

In [ ]:
df_algo_files_myers = []
x = 0
while x < len(diff_files_myers):
    sys.stdout.write("\rExtracting file %i" %(x+1))
    sys.stdout.flush()
    df_algo_files_myers.append([diff_files_myers[x]])
    x += 1
    
df_algo_files_myers

In [ ]:
#Open csv file from result of git blame for <filename>.java
for num, blame in enumerate(blamefiles_myers):
    sys.stdout.write('\rComparing bugline_files and blame_files: %i' % (num+1) + ' out of %i' % (len(blamefiles_myers)))
    sys.stdout.flush()
    
    fields = ['line_number', 'code', 'date']
    blamefile = pd.read_csv(blame)
    blamefile = blamefile[fields]

    origin_code = []
    for n in range(0, len(blamefile)):
        temp = [blamefile.iloc[n][0], str(blamefile.iloc[n][1]).replace('nan',''), blamefile.iloc[n][2]]
        origin_code.append(temp)
    
    #Open diff_file <filename>.java-HBASE-xxxx_<diffalgorithm>buglines.diff vs blame_<filename>
    for m, df in enumerate(df_algo_files_myers[num]):
        dfname = re.search(("(?:\w+[-]\w+[-]\w+|\w+[-]\w+|\w+)[.]\w.*"), df)
        dfname = dfname.group(0)
        
        diff_file = open(analyze_dir + 'file-diff/buggylines/myers/' + dfname,'r')
        bugline = diff_file.read().split('\n')

        if bugline[len(bugline)-1] == '':
            for i in range(0,len(bugline)-1):
                bugline[i] = bugline[i][1:]
            del bugline[len(bugline)-1]
        else:
            for i in range(0,len(bugline)):
                bugline[i] = bugline[i][1:]
        
        bug_id = re.compile('(HBASE-[0-9]*)')
        bug_id = bug_id.findall(df)
        
        for vers in range(0,len(aff)):
            if bug_id[0] == aff.iloc[vers][0]:
                version = aff.iloc[vers][1]
                daterelease = aff.iloc[vers][2]
                break;
            else:
                version = 'None'
                daterelease = 'None'
        
        num = 0
        blame_compare = []
        org_code = [el[1] for el in origin_code]
        for y, bugs in enumerate(bugline):
            if bugs in org_code:
                for x in range(num, len(origin_code)):                
                    codex = origin_code[x][1]
                    if bugs == codex:
                        if version != 'None':
                            if origin_code[x][2] <= daterelease:
                                flag = 'bug-introducing change'
                            else:
                                flag = 'incorrect'
                        else:
                            flag = 'unknown'
                        temp = [bug_id[0], origin_code[x][0], bugs, "True", origin_code[x][2], version, daterelease, flag]
                        blame_compare.append(temp)
                        num = origin_code[x][0]
                        break;
            else:
                temp = [bug_id[0], 'None', bugs, "False", 'None', version, daterelease, 'unknown']
                blame_compare.append(temp)
                #break;
        
        fnames = re.search(("(?:\w+[-]\w+[-]\w+|\w+[-]\w+|\w+)[.]\w+[_]|(?<=myers\/)(\w+[_])(?<=[_])"), df)
        fnames = fnames.group(0)
        
        #algoname = re.search(("[_]\w+.diff"), df)
        #algoname = algoname.group(0)
        
        blamename = re.search(("(?:\w+[-]\w+[-]\w+|\w+[-]\w+|\w+)[.]\w.*"), blame)
        blamename = blamename.group(0)
        
        with open (analyze_dir + r'step_3/diff-file_blame-file_comparison/myers_comparison/diff_' + fnames + "myers-" + blamename, 'w') as csvfile:
            writers = csv.writer(csvfile, delimiter = ",")
            header = ['bug_id', 'line_number', 'buggy_code', 'does_bugline_from_originfile?', 'date_added_in_originfile', 'earliest_affected_version', 'version_release_date', 'bug_class']
            writers.writerow(header)
            for item in blame_compare:
                writers.writerow(item)
                
print ('\nComparing buglines are complete')

# Patience

In [ ]:
filelist_pat = glob.glob(analyze_dir + 'step_3/blame_files/blame_patience/csv/*')
blamefiles_pat = ';'.join(filelist_pat).split(';')
blamefiles_pat.sort()
print ("Found " + str(len(blamefiles_pat)) + " files")

In [ ]:
filebug_pat = glob.glob(analyze_dir + 'file-diff/buggylines/patience/*')
diff_files_pat = ';'.join(filebug_pat).split(';')
diff_files_pat.sort()
print ("Found " + str(len(diff_files_pat)) + " files")

In [ ]:
h = 1034
print (blamefiles_pat[h])
print ("\n" + diff_files_pat[h])

In [ ]:
df_algo_files_pat = []
x = 0
while x < len(diff_files_pat):
    sys.stdout.write("\rExtracting file %i" %(x+1))
    sys.stdout.flush()
    df_algo_files_pat.append([diff_files_pat[x]])
    x += 1
    
df_algo_files_pat

In [ ]:
#Open csv file from result of git blame for <filename>.java
for num, blame in enumerate(blamefiles_pat):
    sys.stdout.write('\rComparing bugline_files and blame_files: %i' % (num+1) + ' out of %i' % (len(blamefiles_pat)))
    sys.stdout.flush()
    
    fields = ['line_number', 'code', 'date']
    blamefile = pd.read_csv(blame)
    blamefile = blamefile[fields]

    origin_code = []
    for n in range(0, len(blamefile)):
        temp = [blamefile.iloc[n][0], str(blamefile.iloc[n][1]).replace('nan',''), blamefile.iloc[n][2]]
        origin_code.append(temp)
    
    #Open diff_file <filename>.java-HBASE-xxxx_<diffalgorithm>buglines.diff vs blame_<filename>
    for m, df in enumerate(df_algo_files_pat[num]):
        dfname = re.search(("(?:\w+[-]\w+[-]\w+|\w+[-]\w+|\w+)[.]\w.*"), df)
        dfname = dfname.group(0)
        
        diff_file = open(analyze_dir + 'file-diff/buggylines/patience/' + dfname,'r')
        bugline = diff_file.read().split('\n')

        if bugline[len(bugline)-1] == '':
            for i in range(0,len(bugline)-1):
                bugline[i] = bugline[i][1:]
            del bugline[len(bugline)-1]
        else:
            for i in range(0,len(bugline)):
                bugline[i] = bugline[i][1:]
        
        bug_id = re.compile('(HBASE-[0-9]*)')
        bug_id = bug_id.findall(df)
        
        for vers in range(0,len(aff)):
            if bug_id[0] == aff.iloc[vers][0]:
                version = aff.iloc[vers][1]
                daterelease = aff.iloc[vers][2]
                break;
            else:
                version = 'None'
                daterelease = 'None'
        
        num = 0
        blame_compare = []
        org_code = [el[1] for el in origin_code]
        for y, bugs in enumerate(bugline):
            if bugs in org_code:
                for x in range(num, len(origin_code)):                
                    codex = origin_code[x][1]
                    if bugs == codex:
                        if version != 'None':
                            if origin_code[x][2] <= daterelease:
                                flag = 'bug-introducing change'
                            else:
                                flag = 'incorrect'
                        else:
                            flag = 'unknown'
                        temp = [bug_id[0], origin_code[x][0], bugs, "True", origin_code[x][2], version, daterelease, flag]
                        blame_compare.append(temp)
                        num = origin_code[x][0]
                        break;
            else:
                temp = [bug_id[0], 'None', bugs, "False", 'None', version, daterelease, 'unknown']
                blame_compare.append(temp)
                #break;
        
        fnames = re.search(("(?:\w+[-]\w+[-]\w+|\w+[-]\w+|\w+)[.]\w+[_]|(?<=patience\/)(\w+[_])(?<=[_])"), df)
        fnames = fnames.group(0)
        
        #algoname = re.search(("[_]\w+.diff"), df)
        #algoname = algoname.group(0)
        
        blamename = re.search(("(?:\w+[-]\w+[-]\w+|\w+[-]\w+|\w+)[.]\w.*"), blame)
        blamename = blamename.group(0)
        
        with open (analyze_dir + r'step_3/diff-file_blame-file_comparison/patience_comparison/diff_' + fnames + "patience-" + blamename, 'w') as csvfile:
            writers = csv.writer(csvfile, delimiter = ",")
            header = ['bug_id', 'line_number', 'buggy_code', 'does_bugline_from_originfile?', 'date_added_in_originfile', 'earliest_affected_version', 'version_release_date', 'bug_class']
            writers.writerow(header)
            for item in blame_compare:
                writers.writerow(item)
                
print ('\nComparing buglines are complete')